In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import *
import math
from tqdm import tqdm

In [ ]:
# Reading offshore wind speed data from csv file
wind_offshore_db = pd.read_csv(r'rawData\offshore_output\hourly_offshore_wind_speed.csv')

# extending it to 30 years
wind_offshore_db = pd.concat([wind_offshore_db, wind_offshore_db, wind_offshore_db], ignore_index=True)

In [ ]:
for d in tqdm(range(3650*3)):
    for h in range(24):
        wind_offshore_db.loc[(d*24)+h, "dayIndex"] = d

representativeDays = [149, 336, 1255, 1381, 1414, 1486, 2239, 3284, 3462]
listHours = []

for y in range(3):
    for d in representativeDays:
        for h in range(24):
            listHours.append(24*(d+y*3650) + h)

wind_offshore_db = wind_offshore_db[wind_offshore_db.index.isin(listHours)]
wind_offshore_db = wind_offshore_db.reset_index(drop=True)

In [ ]:
three_hourly_offshore_wind_speed = pd.DataFrame(columns=["Value"])

emptyListValue = []
emptyListDay = []

for i in tqdm(range(int(len(wind_offshore_db)/3))):
    day = wind_offshore_db.loc[1+(3*i),"dayIndex"]
    value = wind_offshore_db.loc[1+(3*i),"windspeed(m/s)"]
    emptyListValue.append(value)
    emptyListDay.append(day)
    
three_hourly_offshore_wind_speed["Value"] = emptyListValue
three_hourly_offshore_wind_speed["dayIndex"] = emptyListDay

In [ ]:
offshore_wind_values = [0,1,2,3,  4,  5,  6,   7,   8,   9,   10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29]
offshore_power_curve = [0,0,0,157,500,981,1639,2529,3640,4928,6050,6156,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300]

offshore_output = pd.DataFrame(columns=["Value", "dayIndex"])
emptyListValue = []

for i in tqdm(range(len(wind_offshore_db))):
    windSpeed = wind_offshore_db.loc[i,"windspeed(m/s)"]
    windOutput = round(np.interp(windSpeed, offshore_wind_values, offshore_power_curve),3)
    
    emptyListValue.append(windOutput)
    
offshore_output["Value"] = emptyListValue
#offshore_output["dayIndex"] = three_hourly_offshore_wind_speed["dayIndex"]
# finding the output of 1kW installation: 
offshore_output["Value"] = offshore_output.loc[:,"Value"] / max(offshore_power_curve)

In [ ]:
offshore_wind_values = [0,1,2,3,  4,  5,  6,   7,   8,   9,   10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29]
offshore_power_curve = [0,0,0,157,500,981,1639,2529,3640,4928,6050,6156,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300,6300]

offshore_output = pd.DataFrame(columns=["Value", "dayIndex"])
emptyListValue = []

for i in tqdm(range(len(three_hourly_offshore_wind_speed))):
    windSpeed = three_hourly_offshore_wind_speed.loc[i,"Value"]
    windOutput = round(np.interp(windSpeed, offshore_wind_values, offshore_power_curve),3)*3
    
    emptyListValue.append(windOutput)
    
offshore_output["Value"] = emptyListValue
offshore_output["dayIndex"] = three_hourly_offshore_wind_speed["dayIndex"]
# finding the output of 1kW installation: 
offshore_output["Value"] = offshore_output.loc[:,"Value"] / max(offshore_power_curve)

In [ ]:
daily_offshore_windspeed = pd.DataFrame(columns=["Value", "dayIndex"])
emptyList = []

for i in tqdm(range(int(len(offshore_output)/8))):
    daily_offshore_windspeed.loc[i, "Value"] = list(offshore_output.loc[(0 + 8*i):(7 + 8*i), "Value"])
    daily_offshore_windspeed.loc[i, "dayIndex"] = offshore_output.loc[(0 + 8*i), "dayIndex"]

daily_offshore_windspeed.to_json(r"processedData\processed_offshore_output\daily_3hourly_offshoreOutput.json", orient='split')